# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 8 2022 12:43PM,245976,10,8500831,MedStone Pharma LLC,Released
1,Sep 8 2022 12:43PM,245976,10,8500832,MedStone Pharma LLC,Released
2,Sep 8 2022 12:43PM,245976,10,8500833,MedStone Pharma LLC,Released
3,Sep 8 2022 12:43PM,245976,10,8500834,MedStone Pharma LLC,Released
4,Sep 8 2022 12:43PM,245976,10,8500835,MedStone Pharma LLC,Released
5,Sep 8 2022 12:43PM,245976,10,8500836,MedStone Pharma LLC,Released
6,Sep 8 2022 12:43PM,245976,10,8500837,MedStone Pharma LLC,Released
7,Sep 8 2022 12:43PM,245976,10,8500838,MedStone Pharma LLC,Released
8,Sep 8 2022 12:43PM,245976,10,8500839,MedStone Pharma LLC,Released
9,Sep 8 2022 12:43PM,245976,10,8500840,MedStone Pharma LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,245972,Released,2
37,245973,Released,1
38,245974,Released,1
39,245975,Released,1
40,245976,Released,10


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Executing,Released
id,,,
245972,NaN,NaN,2.0
245973,NaN,NaN,1.0
245974,NaN,NaN,1.0
245975,NaN,NaN,1.0
245976,NaN,NaN,10.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Executing,Released
id,,,
245650,0.0,1.0,0.0
245758,1.0,0.0,0.0
245795,0.0,0.0,1.0
245889,0.0,1.0,0.0
245895,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Cancelled,Executing,Released
id,,,
245650,0.0,1,0
245758,1.0,0,0
245795,0.0,0,1
245889,0.0,1,0
245895,0.0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Executing,Released
0,245650,0.0,1,0
1,245758,1.0,0,0
2,245795,0.0,0,1
3,245889,0.0,1,0
4,245895,0.0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Cancelled,Executing,Released
0,245650,0.0,1,
1,245758,1.0,,
2,245795,0.0,,1
3,245889,0.0,1,
4,245895,0.0,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 8 2022 12:43PM,245976,10,MedStone Pharma LLC
10,Sep 8 2022 12:42PM,245975,16,Sartorius Bioprocess Solutions
11,Sep 8 2022 12:41PM,245974,10,ISDIN Corporation
12,Sep 8 2022 12:39PM,245973,10,"Snap Medical Industries, LLC"
13,Sep 8 2022 12:37PM,245972,10,Emerginnova
15,Sep 8 2022 12:32PM,245971,10,Beach Products Inc
31,Sep 8 2022 12:25PM,245968,10,"Methapharm, Inc."
46,Sep 8 2022 12:25PM,245969,15,"Brookfield Pharmaceuticals, LLC"
60,Sep 8 2022 12:20PM,245967,22,"NBTY Global, Inc."
61,Sep 8 2022 12:17PM,245966,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Executing,Released
0,Sep 8 2022 12:43PM,245976,10,MedStone Pharma LLC,0.0,,10
1,Sep 8 2022 12:42PM,245975,16,Sartorius Bioprocess Solutions,0.0,,1
2,Sep 8 2022 12:41PM,245974,10,ISDIN Corporation,0.0,,1
3,Sep 8 2022 12:39PM,245973,10,"Snap Medical Industries, LLC",0.0,,1
4,Sep 8 2022 12:37PM,245972,10,Emerginnova,0.0,,2
5,Sep 8 2022 12:32PM,245971,10,Beach Products Inc,0.0,,16
6,Sep 8 2022 12:25PM,245968,10,"Methapharm, Inc.",0.0,,15
7,Sep 8 2022 12:25PM,245969,15,"Brookfield Pharmaceuticals, LLC",0.0,,14
8,Sep 8 2022 12:20PM,245967,22,"NBTY Global, Inc.",0.0,,1
9,Sep 8 2022 12:17PM,245966,22,"NBTY Global, Inc.",0.0,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 8 2022 12:43PM,245976,10,MedStone Pharma LLC,10,
1,Sep 8 2022 12:42PM,245975,16,Sartorius Bioprocess Solutions,1,
2,Sep 8 2022 12:41PM,245974,10,ISDIN Corporation,1,
3,Sep 8 2022 12:39PM,245973,10,"Snap Medical Industries, LLC",1,
4,Sep 8 2022 12:37PM,245972,10,Emerginnova,2,
5,Sep 8 2022 12:32PM,245971,10,Beach Products Inc,16,
6,Sep 8 2022 12:25PM,245968,10,"Methapharm, Inc.",15,
7,Sep 8 2022 12:25PM,245969,15,"Brookfield Pharmaceuticals, LLC",14,
8,Sep 8 2022 12:20PM,245967,22,"NBTY Global, Inc.",1,
9,Sep 8 2022 12:17PM,245966,22,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 8 2022 12:43PM,245976,10,MedStone Pharma LLC,10,
1,Sep 8 2022 12:42PM,245975,16,Sartorius Bioprocess Solutions,1,
2,Sep 8 2022 12:41PM,245974,10,ISDIN Corporation,1,
3,Sep 8 2022 12:39PM,245973,10,"Snap Medical Industries, LLC",1,
4,Sep 8 2022 12:37PM,245972,10,Emerginnova,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 8 2022 12:43PM,245976,10,MedStone Pharma LLC,10.0,NaN
1,Sep 8 2022 12:42PM,245975,16,Sartorius Bioprocess Solutions,1.0,NaN
2,Sep 8 2022 12:41PM,245974,10,ISDIN Corporation,1.0,NaN
3,Sep 8 2022 12:39PM,245973,10,"Snap Medical Industries, LLC",1.0,NaN
4,Sep 8 2022 12:37PM,245972,10,Emerginnova,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 8 2022 12:43PM,245976,10,MedStone Pharma LLC,10.0,0.0
1,Sep 8 2022 12:42PM,245975,16,Sartorius Bioprocess Solutions,1.0,0.0
2,Sep 8 2022 12:41PM,245974,10,ISDIN Corporation,1.0,0.0
3,Sep 8 2022 12:39PM,245973,10,"Snap Medical Industries, LLC",1.0,0.0
4,Sep 8 2022 12:37PM,245972,10,Emerginnova,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5164670,190.0,17.0
15,491910,78.0,25.0
16,491873,5.0,0.0
18,245934,0.0,1.0
19,491878,3.0,0.0
20,737737,11.0,1.0
21,983359,3.0,1.0
22,737897,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5164670,190.0,17.0
1,15,491910,78.0,25.0
2,16,491873,5.0,0.0
3,18,245934,0.0,1.0
4,19,491878,3.0,0.0
5,20,737737,11.0,1.0
6,21,983359,3.0,1.0
7,22,737897,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,190.0,17.0
1,15,78.0,25.0
2,16,5.0,0.0
3,18,0.0,1.0
4,19,3.0,0.0
5,20,11.0,1.0
6,21,3.0,1.0
7,22,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,190.0
1,15,Released,78.0
2,16,Released,5.0
3,18,Released,0.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Executing,17.0,25.0,0.0,1.0,0.0,1.0,1.0,0.0
Released,190.0,78.0,5.0,0.0,3.0,11.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Executing,17.0,25.0,0.0,1.0,0.0,1.0,1.0,0.0
1,Released,190.0,78.0,5.0,0.0,3.0,11.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Executing,17.0,25.0,0.0,1.0,0.0,1.0,1.0,0.0
1,Released,190.0,78.0,5.0,0.0,3.0,11.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()